# Condo Ownership vs Stock Index Investment

In [114]:
import numpy as np

## Yearly Expenditures for Condo Ownership (3494 hotel-de-ville)

(Property Tax for [Plateau Mont-Royale](http://ville.montreal.qc.ca/pls/portal/docs/PAGE/SERVICE_FIN_EN/MEDIA/DOCUMENTS/2019_PLATEAU_ANG.PDF)) * (Most recent property evaluation for 3494 hotel-de-ville)


In [115]:
plateau_tax_rate = 0.006519 + 0.000025 + 0.001083 + 0.000036 + 0.000591 + 0.000315
municipal_evaluation = 359000
property_tax = plateau_tax_rate * municipal_evaluation

In [116]:
condo_fees = 2000
insurance = 400
maintenance = 300

In [117]:
annual_expense = sum([
    condo_fees,
    insurance,
    maintenance,
    property_tax
])
annual_expense

5776.270999999999

## One-time Expenses for Condo Ownership

In [118]:
welcome_tax = 4500
notary = 1500
title_insurance = 400
inspection = 400

initial_expense = sum([welcome_tax, notary, title_insurance, inspection])
initial_expense

6800

## Mortgage Analysis

In [119]:
interest = 0.03
nper = 25
per = np.arange(nper) + 1
price = 387000
downpayment = 20000

### CMHC Insurance

Interest rates [listed here](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/mortgage-loan-insurance-homeownership-programs/cmhc-mortgage-loan-insurance-cost). Mortgage insurance is covered by the lender for loan to value of under 80% (i.e., if your downpayment is at least 20%, this section is of no concern to you).

In [120]:
def mortgage_insurance_rate(p, downpayment):
    ltv = (p - downpayment) / p
    assert ltv > 0
    assert ltv <= 1
    
    if ltv >= 0.95:
        return 0.04
    elif ltv >= 0.90:
        return 0.031
    elif ltv >= 0.85:
        return 0.028
    elif ltv >= 0.80:
        return 0.024
    else:
        return 0
    
mortgage_insurance_rate(price, downpayment)

0.031

In [121]:
initial_principal = price + mortgage_insurance_rate(price, downpayment) * price - downpayment
ppmt = -np.ppmt(interest, per, nper, price)
ipmt = -np.ipmt(interest, per, nper, price)
pmt = ipmt + ppmt

In [122]:
fmt = '{0:2d} {4:8.2f} {1:8.2f} {2:8.2f} {3:8.2f}'
p = initial_principal
for payment in per:
    index = payment - 1
    p = p + ppmt[index]
    print(fmt.format(payment, ppmt[index], ipmt[index], p, ppmt[index] + ipmt[index]))

# Monthly mortgage payment
(ppmt[0] + ipmt[0]) / 12


 1 22224.59 10614.59 11610.00 389611.59
 2 22224.59 10933.02 11291.56 400544.61
 3 22224.59 11261.01 10963.57 411805.62
 4 22224.59 11598.84 10625.74 423404.47
 5 22224.59 11946.81 10277.78 435351.28
 6 22224.59 12305.21  9919.37 447656.49
 7 22224.59 12674.37  9550.22 460330.86
 8 22224.59 13054.60  9169.98 473385.47
 9 22224.59 13446.24  8778.35 486831.71
10 22224.59 13849.63  8374.96 500681.33
11 22224.59 14265.12  7959.47 514946.45
12 22224.59 14693.07  7531.52 529639.52
13 22224.59 15133.86  7090.72 544773.38
14 22224.59 15587.88  6636.71 560361.26
15 22224.59 16055.51  6169.07 576416.77
16 22224.59 16537.18  5687.41 592953.95
17 22224.59 17033.29  5191.29 609987.25
18 22224.59 17544.29  4680.29 627531.54
19 22224.59 18070.62  4153.96 645602.16
20 22224.59 18612.74  3611.85 664214.90
21 22224.59 19171.12  3053.46 683386.03
22 22224.59 19746.26  2478.33 703132.28
23 22224.59 20338.64  1885.94 723470.93
24 22224.59 20948.80  1275.78 744419.73
25 22224.59 21577.27   647.32 765997.00


1852.0488410118705

### Comparative Yearly Return

Condominium appreciation 
[Plateau Mont Royale centris](https://www.centris.ca/en/tools/real-estate-statistics/montreal-island/le-plateau-mont-royal-montreal) -- 8% last year

[Montreal shupilov.com](https://news.shupilov.com/blog/average-real-estate-prices-and-appreciation-rates-in-montreal/) -- 3% per year

[fciq.ca](https://www.fciq.ca/pdf/mot_economiste/me_052014_an.pdf)



In [123]:
rental_income = 2200 * 12
initial_capital = annual_expense + initial_expense + downpayment
total_yearly_cost = annual_expense + pmt[0]
yearly_capital = total_yearly_cost - rental_income

appreciated_value = np.fv(0.05, per, 0, -price)

cumulative_capital = initial_capital + np.matmul([yearly_capital]*nper, np.triu(np.ones(nper*nper).reshape((nper, nper))))
cumulative_principal_paid = np.matmul(ppmt, np.triu(np.ones(nper*nper).reshape((nper, nper))))
cumulative_principal_left = initial_principal - cumulative_principal_paid

(initial_capital, yearly_capital)

(32576.271, 1600.857092142447)

In [124]:
# TODO: add (varying per year) (annual_expense + ipmt[0] - rental_income * 12) compounded by 1.07
# This would be the lost opportunity cost that you'd be otheriwse adding to your stock investments
# (e.g., what is the added capital beyond your initial investment that you must use to sustain your mortgage?)
stocks = np.fv(0.07, per, -yearly_capital, -initial_capital)
stocks

array([ 36457.46706214,  40610.34684863,  45053.92822018,  49808.56028774,
        54896.01660002,  60339.59485416,  66164.2235861 ,  72396.57632927,
        79065.19376446,  86200.61442011,  93835.51452166, 102004.85763032,
       110746.05475659, 120099.13568169, 130106.93227155, 140815.2746227 ,
       152273.20093844, 164533.18209627, 177651.36193515, 191687.81436275,
       206706.81846029, 222777.15284465, 239972.41063592, 258371.33647258,
       278058.1871178 ])

In [125]:
realestate = appreciated_value - cumulative_capital - cumulative_principal_left
realestate

array([   3790.458     ,   33440.12458276,   64433.65687578,
         96831.68835034,  130697.68735691,  166098.09296519,
        203102.45741917,  241783.59553245,  282217.74136547,
        324484.71254298,  368668.08258817,  414855.36166804,
        463138.18616467,  513612.51750713,  566378.8507207 ,
        621542.43317243,  679213.49401591,  739507.48486316,
        802545.33223758,  868453.70238943,  937365.27908422,
       1009419.05500451, 1084760.63743765, 1163542.56895516,
       1245924.6638245 ])

In [126]:
#calculate roi

roi_realestate = (realestate - cumulative_capital) / cumulative_capital
roi_realestate

array([-0.88909372, -0.06534355,  0.72380023,  1.48415688,  2.22069727,
        2.93770808,  3.63892015,  4.32760979,  5.00667956,  5.67872323,
        6.34607846,  7.01086986,  7.67504452,  8.34040141,  9.00861609,
        9.68126137, 10.35982495, 11.04572444, 11.74032023, 12.44492672,
       13.16082208, 13.88925678, 14.63146126, 15.38865264, 16.16204085])

In [127]:
first_year = rental_income * 12 - annual_expense - initial_expense + ppmt[0] - ipmt[0]
def net(year):
    if year == 1:
        return first_year
    return net(year - 1) + rental_income * 12 - annual_expense + ppmt[year] - ipmt[year]

In [128]:
np.fv(0.07, per, -80000, -50000)

array([ 133500.        ,  222845.        ,  318444.15      ,
        420735.2405    ,  530186.707335  ,  647299.77684845,
        772610.76122784,  906693.51451379, 1050162.06052976,
       1203673.40476684, 1367930.54310052, 1543685.68111756,
       1731743.67879578, 1932965.73631149, 2148273.33785329,
       2378652.47150302, 2625158.14450824, 2888919.21462381,
       3171143.55964748, 3473123.6088228 , 3796242.2614404 ,
       4141979.21974123, 4511917.76512312, 4907752.00868173,
       5331294.64928946])